<a href="https://colab.research.google.com/github/SridharK12/Deep-Learning-Project/blob/main/Quora_Embeddings_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# import libraries for reading data, exploring and plotting
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
print(tf.__version__)
from google.colab import drive
drive.mount('/content/drive')  

2.8.0
Mounted at /content/drive


In [5]:
vocab_size = 20000     #Number of unique words in vocabulary  #Dimensionality of embedding
trunc_type = 'post'    #When the truncation type is post characters to the end of text are truncated whenever the length reaches max length
padding_type = 'post'  #When the truncation type is pre initial characters are deleted when length of text exceeds max length 
oov_tok = '<OOV>'      #This is out of vocabulary token
training_portion = .8

In [6]:
data = pd.read_csv("/content/drive/My Drive/quora_train_clean.csv",encoding='iso-8859-1')

In [7]:
data['target'].value_counts()

0    61278
1     4028
Name: target, dtype: int64

In [8]:
#rows = data.sample(frac =.1)

In [9]:
#rows['target'].value_counts()

In [10]:
#data=rows

In [11]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(data,  test_size = 0.2,random_state=2)

In [12]:
df_train.drop(['Unnamed: 0', 'qid', 'question_text', 'text_length'], axis=1, inplace = True) 

In [13]:
df_test.drop(['Unnamed: 0', 'qid', 'question_text', 'text_length'], axis=1, inplace = True) 

In [14]:
df_train_1 = df_train[df_train['target'] == 1]
df_train_0 = df_train[df_train['target'] == 0]

In [15]:
#Code for upsampling
from sklearn.utils import resample
from sklearn.utils import shuffle
df_train_1_upsampled = resample(df_train_1, 
                                 replace=True,     # sample with replacement#
                                 n_samples= round(0.25*(df_train_0.shape[0])),    # to match majority class
                                 random_state=123) # reproducible results

In [16]:
df_train = pd.concat([df_train_0, df_train_1_upsampled])

In [17]:
df_train=shuffle(df_train)

In [18]:
df_train.shape

(61342, 2)

In [19]:
df_train["target"].value_counts()

0    49074
1    12268
Name: target, dtype: int64

In [20]:
df_test.shape

(13062, 2)

In [21]:
df_train.columns

Index(['target', 'cleaned_text'], dtype='object')

In [22]:
df_test.columns

Index(['target', 'cleaned_text'], dtype='object')

In [23]:
df_test.shape

(13062, 2)

In [24]:
df_train.shape

(61342, 2)

In [25]:
#For backup
df_train_target=df_train['target'].values

In [26]:
#For backup
df_test_target=df_test['target'].values

In [27]:
df_train.drop(['target'], axis=1, inplace=True)

In [28]:
df_test.drop(['target'], axis=1, inplace=True)

In [29]:
df_train.head()

,cleaned_text
14482,husband knows im ok looking porn would clear h...
10330,requirements university houston program reason...
3878,hole crockpot mean
16052,manic defence relates bipolar disorder
62169,quota elitist culture


In [30]:
max_length=300

In [31]:
tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token = oov_tok)
tokenizer.fit_on_texts(df_train['cleaned_text'].astype(str))
training_sequences = tokenizer.texts_to_sequences(df_train['cleaned_text'].astype(str))
training_padded = pad_sequences(training_sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type )

In [32]:
training_padded.shape

(61342, 300)

In [33]:
tokenizer.fit_on_texts(df_test['cleaned_text'].astype(str))
testing_sequences = tokenizer.texts_to_sequences(df_test['cleaned_text'].astype(str))
testing_padded = pad_sequences(testing_sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type )

In [34]:
testing_padded.shape

(13062, 300)

In [35]:
df_train.columns

Index(['cleaned_text'], dtype='object')

In [36]:
#This code is taken from sms classifier
# Map ham label as 0 and spam as 1
#msg_df['msg_type']= msg_df['label'].map({'ham': 0, 'spam': 1})
#msg_label = msg_df['msg_type'].values
# Split data into train and test
#train_msg, test_msg, train_labels, test_labels = train_test_split(msg_df['message'], msg_label, test_size=0.2, random_state=434)

In [37]:
#Code for testing
#data.shape

In [38]:
#data_bkp=data.head(20)

In [39]:
#data_bkp['target'].value_counts()

In [40]:
#data_bkp.drop(['Unnamed: 0', 'qid', 'question_text', 'text_length'], axis=1, inplace=True)

In [41]:
#data_bkp

In [42]:
# Split data into train and test
#train_q, test_q, train_targets, test_targets = train_test_split(data_bkp['cleaned_text'], data_bkp['target'], test_size=0.2, random_state=434)

In [43]:
#train_q

In [44]:
#train_targets

In [45]:
#train_targets.values

In [46]:
#tokenizer.fit_on_texts(train_q)

In [47]:
#training_padded.shape

In [48]:
#training_padded

In [49]:
#training_sequences = tokenizer.texts_to_sequences(train_q)
#training_padded = pad_sequences (training_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type )

In [50]:
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_2(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 5*((precision*recall)/(4*precision+recall+K.epsilon()))

In [51]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2022-04-24 06:22:02--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2022-04-24 06:22:02--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2022-04-24 06:22:03--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [52]:
!unzip glove*.zip
!ls glove*

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      
glove.42B.300d.txt  glove.42B.300d.zip


https://sabber.medium.com/classifying-yelp-review-comments-using-cnn-lstm-and-pre-trained-glove-word-embeddings-part-3-53fcea9a17fa
https://medium.com/analytics-vidhya/text-classification-using-word-embeddings-and-deep-learning-in-python-classifying-tweets-from-6fe644fcfc81
https://keras.io/examples/nlp/pretrained_word_embeddings/

https://blog.paperspace.com/pre-trained-word-embeddings-natural-language-processing/
https://towardsdatascience.com/pre-trained-word-embedding-for-text-classification-end2end-approach-5fbf5cd8aead

In [53]:
embeddings_index = dict()
f = open('glove.42B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')    
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1917494 word vectors.


In [54]:
max_length=300

In [55]:
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, max_length))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [56]:
embedding_matrix.shape

(37070, 300)

In [57]:
import numpy as np
import tensorflow
class EarlyStoppingAtMinLoss(tensorflow.keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.
  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """
    def __init__(self, patience=0):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None
    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))

#model.evaluate(testing_padded, df_test_target)

In [58]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1,
                            output_dim=max_length,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

In [60]:
from tensorflow.keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
model=Sequential()

#embedding=embedding_layer
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(LSTM(300, dropout=0.2))
#model.add(LSTM(300, dropout=0.2))
#model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
optimizer = Adam(clipvalue=0.5) # clip value to avoid the gradient exploding
#optimizer=Adam()

model.compile(optimizer=optimizer, 
              loss='binary_crossentropy', 
              metrics=['acc',f1_2,precision_m, recall_m])


In [ ]:
#from tensorflow.keras.models import Sequential
#model = Sequential([
#    embedding_layer,
#    Bidirectional(LSTM(150, return_sequences=True, input_shape=(3,1))), 
#    Bidirectional(LSTM(150, dropout=0.5)),
#    Dense(128, activation='relu'),
#    Dense(1, activation='sigmoid')
#])

In [ ]:
#from tensorflow.keras.optimizers import Adam
#model.compile(optimizer='Adam', 
#              loss='binary_crossentropy', 
#              metrics=['acc',f1_2,precision_m, recall_m])


In [ ]:
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
#%load_ext tensorboard
##rm -rf logs

#log_folder = 'logs'
#callbacks = [
#            EarlyStopping(patience = 10),
#            TensorBoard(log_dir=log_folder)
#            ]
#num_epochs = 600


In [ ]:
#model.fit(training_padded, df_train_target, epochs=10, validation_data=(testing_padded, df_test_target))

#model.fit(training_padded, df_train_target, epochs=5, validation_data=(testing_padded, df_test_target))

In [61]:
model.fit(training_padded, df_train_target,  batch_size=200, epochs=100, validation_data=(testing_padded, df_test_target),
          callbacks=[EarlyStoppingAtMinLoss()]
          )#, verbose=2)

Epoch 1/100
307/307 [==============================] - 1790s 6s/step - loss: 0.5086 - acc: 0.8000 - f1_2: 4.7799e-04 - precision_m: 0.0052 - recall_m: 3.9219e-04 - val_loss: 0.3229 - val_acc: 0.9343 - val_f1_2: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/100
307/307 [==============================] - 1789s 6s/step - loss: 0.5018 - acc: 0.8000 - f1_2: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 0.3139 - val_acc: 0.9343 - val_f1_2: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/100
307/307 [==============================] - 1775s 6s/step - loss: 0.5012 - acc: 0.8000 - f1_2: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 0.3121 - val_acc: 0.9343 - val_f1_2: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 4/100
307/307 [==============================] - 1776s 6s/step - loss: 0.5009 - acc: 0.8000 - f1_2: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 

In [ ]:
model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=30,
    verbose=0,
    callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()],
)

In [ ]:
import numpy as np
class EarlyStoppingAtMinLoss(keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.
  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """
    def __init__(self, patience=0):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None
    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))

model.evaluate(testing_padded, df_test_target)

In [ ]:
training_padded.shape

In [ ]:
df_train_target.shape

In [ ]:
testing_padded.shape